In [3]:
import fxcmpy
import socketio
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
import itertools
from datetime import datetime as dt
import pytz
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
import requests
import json
from scipy.signal import argrelextrema

TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')

ServerError: Can not connect to FXCM Server.

In [4]:
##  INITIAL SETUP

## WATCHLIST = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']

pair = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']
PAIR = []
ICHMKU = []
RSIR = []
RSID = []
RFP = []
TRNGL = []
CANDLE = []
h = len(pair)



## FIRST, WE HAVE TO CHECK WHAT THE CURRENT TREND IS DOING
## WE WILL CONSIDER THE REGRESSION LOBF FOR THE LAST 2 WEEKS (10 DAYS = 60 CANDLES)

with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_I/MarkI_Results.xlsx') as writerdf:
    with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_I/MarkI_Levels.xlsx') as writerdft:
        with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_I/MarkI_Candles.xlsx') as writerdfd:
    
            for x in pair:
                S = con.get_candles(x, period='H4', number=60)
                S = S.tz_localize('Etc/GMT-4')
                S = S.tz_convert('UTC')
                S['open'] = (S['bidopen']+S['askopen'])/2
                S['close'] = (S['bidclose']+S['askclose'])/2
                S['high'] = (S['bidhigh']+S['askhigh'])/2
                S['low'] = (S['bidlow']+S['asklow'])/2

                S = S.assign(MOVE=(S['close'] - S['open']))
                AvgCandle = np.average(abs(S['MOVE']))

                CP = S['close'].iloc[-1]

                l = int(S['close'].count())

                x1 = np.linspace(1, l, num=l)
                y1 = S.iloc[:,10]
                m = sm.OLS(y1, x1).fit()
                slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
                trend = slope    



            ## ICHIMOKU CONFIDENCE TEST
            ## WE WILL CONTINUE WITH THE 4-HOUR AND 60 CANDLES


                k = int(S['close'].count())
                Sdk = np.linspace(1,k,num=k)

                p = k
                RED = []
                BLUE = []
                GREEN = []
                CLOUDA = []
                CLOUDB = []

                for t in Sdk:
                    if p < 9:
                        RLhigh = 1
                        RLlow = 1
                    else:
                        RL = S[p-9:p]
                        RLhigh = max(RL['high'])
                        RLlow = min(RL['low'])
                    redline = (RLhigh + RLlow)/2
                    RED.append(redline)

                    if p < 26:
                        BLhigh = 1
                        BLlow = 1
                    else:
                        BL = S[p-26:p]
                        BLhigh = max(BL['high'])
                        BLlow = min(BL['low'])
                    blueline = (BLhigh + BLlow)/2
                    BLUE.append(blueline)

                    greenline = BL['close'][0]
                    GREEN.append(greenline)

                    if p < 52:
                        CLhigh = 1
                        CLlow = 1
                    else:
                        CL = S[p-52:p]
                        CLhigh = max(CL['high'])
                        CLlow = min(CL['low'])
                    clouda = (redline+ blueline)/2
                    cloudb = (CLhigh + CLlow)/2
                    CLOUDA.append(clouda)
                    CLOUDB.append(cloudb)

                    p = p - 1

                CLOUDA.reverse() 
                CLOUDB.reverse() 
                cloud = [i / j for i, j in zip(CLOUDA, CLOUDB)]
                cloud = [x-1 for x in cloud]
                S['Cloud'] = cloud
                pclouda = S['close']/CLOUDA - 1
                S['PvCloudA'] = pclouda
                pcloudb = S['close']/CLOUDB - 1
                S['PvCloudB'] = pcloudb

                RED.reverse()
                BLUE.reverse() 
                redblue = [i / j for i, j in zip(RED, BLUE)]
                redblue = [x-1 for x in redblue]
                S['RedvBlue'] = redblue
                pblue = S['close']/BLUE - 1
                S['PvBlue'] = pblue

                GREEN.reverse() 
                pgreen = S['close']/GREEN - 1
                S['PvGreen'] = pgreen

                S['BLUE'] = BLUE
                S = S.assign(BluevLast=((S['BLUE'] - S['BLUE'].shift())/S['BLUE']))

                if trend > 0:
                    Igrade_cloud = cloud[-1]*100
                    Igrade_pclouda = pclouda[-1]*100
                    Igrade_pcloudb = pcloudb[-1]*100
                    Igrade_redblue = redblue[-1]*100
                    Igrade_pblue = pblue[-1]*100
                    Igrade_pgreen = pgreen[-1]*100
                    Igrade_blue = S['BluevLast'][-1]*100

                elif trend < 0:
                    Igrade_cloud = cloud[-1]*-100
                    Igrade_pclouda = pclouda[-1]*-100
                    Igrade_pcloudb = pcloudb[-1]*-100
                    Igrade_redblue = redblue[-1]*-100
                    Igrade_pblue = pblue[-1]*-100
                    Igrade_pgreen = pgreen[-1]*-100
                    Igrade_blue = S['BluevLast'][-1]*-100  

                K = list(S['Cloud'][k-52::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_cloud2 = (sum_gains/(sum_gains+sum_losses))


                K = list(S['PvCloudA'][k-52::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_pclouda2 = (sum_gains/(sum_gains+sum_losses))


                K = list(S['PvCloudB'][k-52::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_pcloudb2 = (sum_gains/(sum_gains+sum_losses))


                K = list(S['RedvBlue'][k-26::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_redblue2 = (sum_gains/(sum_gains+sum_losses))

                K = list(S['PvBlue'][k-26::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_pblue2 = (sum_gains/(sum_gains+sum_losses))

                K = list(S['PvGreen'][k-26::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_pgreen2 = (sum_gains/(sum_gains+sum_losses))

                K = list(S['BluevLast'][k-26::])

                Kgains = []
                Klosses = []
                sum_gains = 0
                sum_losses = 1

                if trend > 0:
                    for t in K:
                        if t > 0:
                            Kgains.append(t)
                        elif t < 0:
                            Klosses.append(t)

                        sum_gains = np.sum(Kgains)
                        sum_losses = abs(np.sum(Klosses))
                elif trend < 0:
                    for t in K:
                        if t < 0:
                            Kgains.append(t)
                        elif t > 0:
                            Klosses.append(t)

                        sum_gains = abs(np.sum(Kgains))
                        sum_losses = abs(np.sum(Klosses))

                Igrade_blue2 = (sum_gains/(sum_gains+sum_losses))

                Iw1 = 0.20
                Iw2 = 0.20
                Iw3 = 0.12
                Iw4 = 0.12
                Iw5 = 0.06
                Iw6 = 0.30

                Igrade = Iw1*Igrade_cloud + Iw2*((Igrade_pclouda + Igrade_pcloudb)/2) + Iw3*Igrade_redblue + Iw4*Igrade_pblue + Iw5*Igrade_pgreen + Iw6*Igrade_blue

                Igrade2 = Iw1*Igrade_cloud2 + Iw2*((Igrade_pclouda2 + Igrade_pcloudb2)/2) + Iw3*Igrade_redblue2 + Iw4*Igrade_pblue2 + Iw5*Igrade_pgreen2 + Iw6*Igrade_blue2


                I1w = 0.25
                I2w = 0.75

                ichmku = I1w*Igrade + I2w*Igrade2



            ## RSI REJECTION TEST
            ## STILL USING 4-HOUR CHART WITH 60 CANDLES OF DATA AKA 10 DAYS


                T = S.close.diff()
                window = 14
                up_days = T.copy()
                up_days[T<=0]=0.0
                down_days = abs(T.copy())
                down_days[T>0]=0.0
                RS_up = up_days.rolling(window).mean()
                RS_down = down_days.rolling(window).mean()
                RSIq = 100-100/(1+RS_up/RS_down)
                RSI = RSIq[-1]

                patRSIR = 'None'
                if trend > 0:
                    if RSI < 30:
                        patRSIR = 'RSI rejection for Up-trend'
                elif trend < 0:
                    if RSI > 70:
                        patRSIR = 'RSI rejection for Down-trend'



            ## RSI DIVERGENCE TEST
            ## NOW WE ARE SWITCHING TO THE 8-HOUR CHART WITH 30 CANDLES OF DATA


                S = con.get_candles(x, period='H8', number=30)
                S = S.tz_localize('Etc/GMT-4')
                S = S.tz_convert('UTC')
                S['open'] = (S['bidopen']+S['askopen'])/2
                S['close'] = (S['bidclose']+S['askclose'])/2
                S['high'] = (S['bidhigh']+S['askhigh'])/2
                S['low'] = (S['bidlow']+S['asklow'])/2

                T = S.close.diff()
                window = 14
                up_days = T.copy()
                up_days[T<=0]=0.0
                down_days = abs(T.copy())
                down_days[T>0]=0.0
                RS_up = up_days.rolling(window).mean()
                RS_down = down_days.rolling(window).mean()
                RSIq = 100-100/(1+RS_up/RS_down)
                RSI = RSIq[-1]

                RSIqchange = RSIq - RSIq.shift()
                RSIchange = RSIqchange[-1]

                patRSID = 'None'
                if trend > 0:
                    if RSI < 50:
                        if RSIchange < 0:
                            patRSID = 'RSI Divergence for Up-trend'
                elif trend < 0:
                    if RSI > 50:
                        if RSIchange > 0:
                            patRSID = 'RSI Divergence for Down-trend'



            ## RISING AND FALLING PEAKS TEST
            ## AND HERE WE SWITCH TO THE DAILY WITH 30 CANDLES


                S = con.get_candles(x, period='D1', number=30)
                S = S.tz_localize('Etc/GMT-4')
                S = S.tz_convert('UTC')
                S['open'] = (S['bidopen']+S['askopen'])/2
                S['close'] = (S['bidclose']+S['askclose'])/2
                S['high'] = (S['bidhigh']+S['askhigh'])/2
                S['low'] = (S['bidlow']+S['asklow'])/2

                S = S.assign(MOVE=(S['close'] - S['open']))
                AvgCandle = np.average(abs(S['MOVE']))

                CP = S['close'].iloc[-1]

                l = int(S['close'].count())

                CNDL = (S.iloc[:, 10]) - (S.iloc[:, 9])
                S['CNDL'] = np.where(CNDL > 0, 1, -1)
                S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
                S = S[S.INFL != 0]
                S['INFL'] = S['CNDL']*-1
                S.drop(S.tail(1).index,inplace=True)

                Sh = S[S.INFL != -1]
                Sh = Sh.high - Sh.high.shift()
                Sl = S[S.INFL != 1]
                Sl = Sl.low - Sl.low.shift()

                patRFP = 'None'
                if trend > 0:
                    if Sh[-1] > AvgCandle:
                        if Sh[-2] > AvgCandle:
                            if Sl[-1] > AvgCandle:
                                if Sl[-2] > AvgCandle:
                                    patRFP = 'Rising Highs and Lows Up-trend'      
                elif trend < 0:
                    if Sl[-1] < 0:
                        if Sl[-1] > -1*AvgCandle:
                            if Sl[-2] > -1*AvgCandle:
                                if Sh[-1] > -1*AvgCandle:
                                    if Sh[-2] > -1*AvgCandle:
                                        patRFP = 'Falling Highs and Lows Down-trend'



            ## TRIANGLE FORMATION TEST
            ## STILL USING DAILY


                patTRNGL = 'None'
                if trend > 0:
                    if Sl[-1] > 0:
                        if Sl[-2] > 0:
                            if abs(Sh[-1] - Sh[-2]) < AvgCandle: 
                                    patTRNGL = 'Up-trend Triangle Spotted'     
                elif trend < 0:
                    if Sh[-1] < 0:
                        if Sh[-2] < 0:
                            if abs(Sl[-1] - Sl[-2]) < AvgCandle: 
                                    patTRNGL = 'Down-trend Triangle Spotted'  



            ## NOW WE'LL SET UP THE LEVELS ANALYSIS AND GET READY TO RUN THE BREAK & RETEST TESTS
            ## RETURNING TO THE 4-HOUR AND BUMPING UP TO 1440 CANDLESTICKS AKA 12 MONTHS


                S = con.get_candles(x, period='H4', number=1440)
                S = S.tz_localize('Etc/GMT-4')
                S = S.tz_convert('UTC')
                S['open'] = (S['bidopen']+S['askopen'])/2
                S['close'] = (S['bidclose']+S['askclose'])/2
                S['high'] = (S['bidhigh']+S['askhigh'])/2
                S['low'] = (S['bidlow']+S['asklow'])/2

                S = S.assign(MOVE=(S['close'] - S['open']))
                AvgCandle = np.average(abs(S['MOVE']))
                CP = S['close'].iloc[-1]
                l = int(S['close'].count())

                CPU = CP + AvgCandle*40                        ## THIS IS THE % MARGIN FOR THE SCOPE OR # OF LEVELS
                CPD = CP - AvgCandle*40

                CM = AvgCandle*2                                 ## THIS IS THE % MARGIN FOR THE CONFLUENCE OF LEVELS

                CNDL = (S.iloc[:, 10]) - (S.iloc[:, 9])
                S['CNDL'] = np.where(CNDL > 0, 1, -1)
                S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
                S = S[S.INFL != 0]
                S['INFL'] = S['CNDL']*-1
                S['LVL'] = np.where(S.INFL == 1, S.high, S.low)
                S = S.assign(SIZE=abs(S['LVL'] - (S['LVL'].shift(-1))))

                A = np.array(S.LVL)

                B = list(combinations(A, 2))
                Ba = np.array([x for x, y in B])                       #### HERE'S WHERE TO ROUND THE LEVEL GROUPINGS
                B = np.array([abs(x - y) for x, y in B])
                Bb = np.array([np.where(x <= CM, 1, 0) for x in B])             #### RULE WHERE CONFLUENCE MARGIN COMES INTO PLAY

                T = list(zip(Ba,Bb))

                table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
                table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
                table = table.sort_values(['a'], ascending=False)
                table = table.rename(columns={'b': 'Pings'})
                table = table[table.index <= CPU]
                table = table[table.index >= CPD]
                table = table.fillna(0)
                table = table[table['max'] != 0]

                crank1 = 0                                                  ## CRANK 1 TO ADJUST LEVEL ACCURACY
                tabmean = np.average(table['max'])
                tabstd = np.std(table['max'])
                tablim = tabstd*crank1 + tabmean
                table = table[table['max'] > tablim]

                crank2 = 1                                             ## CRANK 2 TO ADJUST LEVEL ACCURACY
                table = table.assign(SPACE=(table.index))
                table.SPACE = abs(table.SPACE - table.SPACE.shift())      
                spacemean = np.average(table.iloc[1:,2])
                spacestd = np.std(table.iloc[1:,2])
                spacelim = spacestd*crank2 + spacemean
                table = table[table['SPACE'] >= spacelim]



            ## NOW WE WILL SET UP THE CANDLESTICK TABLE COLUMNS
            ## STILL USING THE 4-HOUR AND 60 CANDLESTICKS - 2 WEEKS    


                D = con.get_candles(x, period='H4', number=60)
                D = D.tz_localize('Etc/GMT-4')
                D = D.tz_convert('UTC')
                D['open'] = (D['bidopen']+D['askopen'])/2
                D['close'] = (D['bidclose']+D['askclose'])/2
                D['high'] = (D['bidhigh']+D['askhigh'])/2
                D['low'] = (D['bidlow']+D['asklow'])/2

                D = D.assign(MOVE=(D['close'] - D['open']))
                AvgCandle = np.average(abs(S['MOVE']))
                CAM = AvgCandle*0.15
                CP = D['close'].iloc[-1]
                l = int(D['close'].count())
                lin = np.linspace(1,l,num=l)

                D = D.assign(STICK=(D['high'] - D['low']))
                D = D.assign(PSTICK=abs(D['MOVE']/D['STICK']))
                D['DIRECT'] = np.where(D.MOVE > 0, 'Bullish', 'Bearish')

                ANIMAL = []
                i = 0
                for t in lin: 
                    if D.iloc[i,15] <= 1.1:
                        if D.iloc[i,15] < 0.85:
                            if D.iloc[i,15] < 0.65:
                                if D.iloc[i,15] < 0.50:
                                    if D.iloc[i,15] <= 0.35:
                                        if D.iloc[i,15] <= 0.15:
                                            animal = 'Doji'
                                        else:
                                            animal = 'Frog'
                                    else:
                                        animal = 'Cat'
                                else:
                                    animal = 'Dog'
                            else:
                                animal = 'Horse'
                        else:
                            animal = 'Shark'

                    ANIMAL.append(animal)
                    i = i + 1
                D['ANIMAL'] = ANIMAL

                GAUGE = []
                i = 0
                for t in lin: 
                    cndmid = (D.iloc[i,9]+D.iloc[i,10])/2
                    stickmid = (D.iloc[i,11]+D.iloc[i,12])/2
                    midpoint = (cndmid - stickmid)/D.iloc[i,14]
                    if midpoint >= 0.075:
                        if midpoint >= 0.175:
                            if midpoint >= 0.25:
                                gauge = 'Top Saturated'
                            else:
                                gauge = 'Top Stacked'
                        else:
                            gauge = 'Top Semi-Stacked'
                    elif midpoint <= -0.075:
                        if midpoint <= -0.175:
                            if midpoint <= -0.25:
                                gauge = 'Bottom Saturated'
                            else:
                                gauge = 'Bottom Stacked'
                        else:
                            gauge = 'Bottom Semi-Stacked'
                    else:
                        gauge = 'Centered'

                    GAUGE.append(gauge)
                    i = i + 1
                D['GAUGE'] = GAUGE

                thh = 6                                         ## CANDLESTICK INTERVAL FOR NET TREND (SET TO 6 CANDLES NOW)
                tl = l - thh
                tlin = np.linspace(1,tl,num=tl)
                tlinb = np.linspace(1,thh,num=thh)
                TREND = []
                for t in tlinb:
                    TREND.append(0)
                i = thh                                             
                ii = i - (thh-5)                                          
                for t in tlin: 
                    movetrend = np.average(D.iloc[ii:i,13])
                    if movetrend > 0:
                        mtrend = 1
                    elif movetrend < 0:
                        mtrend = -1
                    else:
                        mtrend = 0
                    i = i + 1
                    TREND.append(mtrend)
                D['TREND'] = TREND

            ## SINGLE-CANDLESTICK FORMATIONS    

                READING=[]
                i = 0
                for t in lin: 
                    if D.iloc[i,19] == -1:
                        if D.iloc[i,18] == 'Top Saturated':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'JACK'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'DIPPER'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'HAMMER'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'DRAGONFLY DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Top Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'JACK'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'SKILLET'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'MALLET'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'DRAGONFLY DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Top Semi-Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'PLUMP DOJI'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'LONG-LEG DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Centered':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'PLUMP DOJI'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'LONG-LEG DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Bottom Semi-Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'PLUMP DOJI'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'LONG-LEG DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Bottom Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'BOOST'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'BRAKE'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'BRAKE'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Bottom Saturated':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'BOOST'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'BRAKE'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'BRAKE'
                            else:
                                Read = D.iloc[i,17]
                        else:
                            Read = D.iloc[i,17]
                    elif D.iloc[i,19] == 1:
                        if D.iloc[i,18] == 'Bottom Saturated':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'JACK'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'DIPPER'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'SHOOTING STAR'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'GRAVESTONE DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Bottom Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'JACK'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'SKILLET'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'ASTEROID'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'GRAVESTONE DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Bottom Semi-Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'PLUMP DOJI'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'LONG-LEG DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Centered':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'PLUMP DOJI'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'LONG-LEG DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Top Semi-Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'PLUMP DOJI'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'LONG-LEG DOJI'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Top Stacked':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'BOOST'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'BRAKE'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'BRAKE'
                            else:
                                Read = D.iloc[i,17]
                        elif D.iloc[i,18] == 'Top Saturated':
                            if D.iloc[i,17] == 'Shark':
                                Read = 'MARIBOZU'
                            elif D.iloc[i,17] == 'Horse':
                                Read = 'TUBE'
                            elif D.iloc[i,17] == 'Dog':
                                Read = 'BOOST'
                            elif D.iloc[i,17] == 'Cat':
                                Read = 'GLIDE'
                            elif D.iloc[i,17] == 'Frog':
                                Read = 'BRAKE'
                            elif D.iloc[i,17] == 'Doji':
                                Read = 'BRAKE'
                            else:
                                Read = D.iloc[i,17]
                        else:
                            Read = D.iloc[i,17]
                    else:
                        Read = D.iloc[i,17]
                    i = i + 1
                    READING.append(Read)
                D['READING'] = READING

            ## DOUBLE-CANDLESTICK FORMATIONS  



                READING2=[]
                READING2.append('--')
                jl = l - 1
                linjj = np.linspace(1,jl,num=jl)
                i = 1
                ii = i - 1
                for t in linjj: 
                    if D.iloc[i,19] == -1:
                        if D.iloc[ii,16] == 'Bearish':
                            if D.iloc[i,10] > D.iloc[ii,11]:
                                if D.iloc[ii,20] == 'BRAKE':
                                    if D.iloc[i,16] == 'Bullish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                elif D.iloc[ii,20] == 'GLIDE':
                                    if D.iloc[i,16] == 'Bullish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                elif D.iloc[ii,20] == 'PLUMP DOJI':
                                    if D.iloc[i,16] == 'Bullish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                elif D.iloc[ii,20] == 'LONG-LEG DOJI':
                                    if D.iloc[i,16] == 'Bullish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                else:
                                    Read2 = '--'
                            elif D.iloc[i,9] == D.iloc[ii,10]:
                                if D.iloc[i,12] == D.iloc[ii,12]:
                                    if D.iloc[i,16] == 'Bullish':
                                        if D.iloc[i,10] > D.iloc[ii,9]:
                                            Read2 = 'TWEEZERS BOTTOM'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                else:
                                    Read2 = '--'
                            elif D.iloc[i,10] > (D.iloc[ii,9]+D.iloc[ii,10])/2:
                                if D.iloc[i,9] < D.iloc[ii,10]:
                                    if D.iloc[ii,16] == 'Bullish':
                                        Read2 = 'PIERCING PATTERN'
                                    else:
                                        Read2 = '--'
                                else:
                                    Read2 = '--'
                            else:
                                Read2 = '--'
                        else:
                            Read2 = '--'
                    elif D.iloc[i,19] == 1:
                        if D.iloc[ii,16] == 'Bullish':
                            if D.iloc[i,10] < D.iloc[ii,12]:
                                if D.iloc[ii,20] == 'BRAKE':
                                    if D.iloc[i,16] == 'Bearish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                elif D.iloc[ii,20] == 'GLIDE':
                                    if D.iloc[i,16] == 'Bearish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                elif D.iloc[ii,20] == 'PLUMP DOJI':
                                    if D.iloc[i,16] == 'Bearish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                elif D.iloc[ii,20] == 'LONG-LEG DOJI':
                                    if D.iloc[i,16] == 'Bearish':
                                        if D.iloc[i,20] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif D.iloc[i,20] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = '--'
                                    else:
                                        Read2 = '--'
                                else:
                                    Read2 = '--'
                            else:
                                Read2 = '--'
                        elif D.iloc[i,10] == D.iloc[ii,9]:
                            if D.iloc[i,11] == D.iloc[ii,11]:
                                if D.iloc[i,16] == 'Bearish':
                                    if D.iloc[i,10] < D.iloc[ii,9]:
                                        Read2 = 'TWEEZERS TOP'
                                    else:
                                        Read2 = '--'
                                else:
                                    Read2 = '--'
                            else:
                                Read2 = '--'
                        elif D.iloc[i,10] < (D.iloc[ii,9]+D.iloc[ii,10])/2:
                            if D.iloc[i,9] > D.iloc[ii,10]:
                                if D.iloc[ii,16] == 'Bearish':
                                    Read2 = 'DARK CLOUD COVER'
                                else:
                                    Read2 = '--'
                            else:
                                Read2 = '--'
                        else:
                            Read2 = '--'
                    else:
                        Read2 = '--'
                    i = i + 1
                    READING2.append(Read2)
                D['READING2'] = READING2

            ## FIVE-CANDLESTICK FORMATIONS

                xhh = 5
                xl = l - xhh
                xlin = np.linspace(1,xl,num=xl)
                xlinb = np.linspace(1,xhh,num=xhh)
                READING3 = []
                for t in xlinb:
                    READING3.append('--')
                i1 = xhh                                             
                i2 = i - 1
                i3 = i - 2  
                i4 = i - 3  
                i5 = i - 4  
                for t in xlin:
                    if D.iloc[i5,16] == 'Bullish':
                        if D.iloc[i5,20] == 'MARIBOZU':
                            if D.iloc[i4,16] == 'Bearish':
                                if D.iloc[i4,10] > (D.iloc[i5,9]+D.iloc[i5,10])/2:
                                    if D.iloc[i3,16] == 'Bearish':
                                        if D.iloc[i3,10] < D.iloc[i4,10]:
                                            if D.iloc[i3,10] > D.iloc[i5,9]:
                                                if D.iloc[i2,16] == 'Bearish':
                                                    if D.iloc[i2,10] < D.iloc[i3,10]:
                                                        if D.iloc[i2,10] > D.iloc[i5,9]:
                                                            if D.iloc[i,16] == 'Bullish':
                                                                if D.iloc[i,20] == 'MARIBOZU':
                                                                    Read3 = 'RISING THREE'
                                                                elif D.iloc[i,20] == 'TUBE':
                                                                    Read3 = 'RISING THREE'
                                                                else:
                                                                    Read3 = '--'
                                                            else:
                                                                Read3 = '--'
                                                        else:
                                                            Read3 = '--'
                                                    else:
                                                        Read3 = '--'
                                                else:
                                                    Read3 = '--'
                                            else:
                                                Read3 = '--'
                                        else:
                                            Read3 = '--'                               
                                    else:
                                        Read3 = '--'
                                else:
                                    Read3 = '--'
                            else:
                                Read3 = '--'
                        elif D.iloc[i5,20] == 'TUBE':
                            if D.iloc[i4,16] == 'Bearish':
                                if D.iloc[i4,10] > (D.iloc[i5,9]+D.iloc[i5,10])/2:
                                    if D.iloc[i3,16] == 'Bearish':
                                        if D.iloc[i3,10] < D.iloc[i4,10]:
                                            if D.iloc[i3,10] > D.iloc[i5,9]:
                                                if D.iloc[i2,16] == 'Bearish':
                                                    if D.iloc[i2,10] < D.iloc[i3,10]:
                                                        if D.iloc[i2,10] > D.iloc[i5,9]:
                                                            if D.iloc[i,16] == 'Bullish':
                                                                if D.iloc[i,20] == 'MARIBOZU':
                                                                    Read3 = 'RISING THREE'
                                                                elif D.iloc[i,20] == 'TUBE':
                                                                    Read3 = 'RISING THREE'
                                                                else:
                                                                    Read3 = '--'
                                                            else:
                                                                Read3 = '--'
                                                        else:
                                                            Read3 = '--'
                                                    else:
                                                        Read3 = '--'
                                                else:
                                                    Read3 = '--'
                                            else:
                                                Read3 = '--'
                                        else:
                                            Read3 = '--'                               
                                    else:
                                        Read3 = '--'
                                else:
                                    Read3 = '--'
                            else:
                                Read3 = '--'
                        else:
                            Read3 = '--'
                    elif D.iloc[i5,16] == 'Bearish':
                        if D.iloc[i5,20] == 'MARIBOZU':
                            if D.iloc[i4,16] == 'Bullish':
                                if D.iloc[i4,10] < (D.iloc[i5,9]+D.iloc[i5,10])/2:
                                    if D.iloc[i3,16] == 'Bullish':
                                        if D.iloc[i3,10] > D.iloc[i4,10]:
                                            if D.iloc[i3,10] < D.iloc[i5,9]:
                                                if D.iloc[i2,16] == 'Bullish':
                                                    if D.iloc[i2,10] > D.iloc[i3,10]:
                                                        if D.iloc[i2,10] < D.iloc[i5,9]:
                                                            if D.iloc[i,16] == 'Bearish':
                                                                if D.iloc[i,20] == 'MARIBOZU':
                                                                    Read3 = 'FALLING THREE'
                                                                elif D.iloc[i,20] == 'TUBE':
                                                                    Read3 = 'FALLING THREE'
                                                                else:
                                                                    Read3 = '--'
                                                            else:
                                                                Read3 = '--'
                                                        else:
                                                            Read3 = '--'
                                                    else:
                                                        Read3 = '--'
                                                else:
                                                    Read3 = '--'
                                            else:
                                                Read3 = '--'
                                        else:
                                            Read3 = '--'                               
                                    else:
                                        Read3 = '--'
                                else:
                                    Read3 = '--'
                            else:
                                Read3 = '--'
                        if D.iloc[i5,20] == 'MARIBOZU':
                            if D.iloc[i4,16] == 'Bullish':
                                if D.iloc[i4,10] < (D.iloc[i5,9]+D.iloc[i5,10])/2:
                                    if D.iloc[i3,16] == 'Bullish':
                                        if D.iloc[i3,10] > D.iloc[i4,10]:
                                            if D.iloc[i3,10] < D.iloc[i5,9]:
                                                if D.iloc[i2,16] == 'Bullish':
                                                    if D.iloc[i2,10] > D.iloc[i3,10]:
                                                        if D.iloc[i2,10] < D.iloc[i5,9]:
                                                            if D.iloc[i,16] == 'Bearish':
                                                                if D.iloc[i,20] == 'MARIBOZU':
                                                                    Read3 = 'FALLING THREE'
                                                                elif D.iloc[i,20] == 'TUBE':
                                                                    Read3 = 'FALLING THREE'
                                                                else:
                                                                    Read3 = '--'
                                                            else:
                                                                Read3 = '--'
                                                        else:
                                                            Read3 = '--'
                                                    else:
                                                        Read3 = '--'
                                                else:
                                                    Read3 = '--'
                                            else:
                                                Read3 = '--'
                                        else:
                                            Read3 = '--'                               
                                    else:
                                        Read3 = '--'
                                else:
                                    Read3 = '--'
                            else:
                                Read3 = '--'
                        else:
                            Read3 = '--'
                    else:
                        Read3 = '--'

                    i = i + 1
                    READING3.append(Read3)
                D['READING3'] = READING3        




            ## BREAK & RETEST OF HISTORICAL LEVEL TEST
            ## STILL USING THE 4-HOUR AND 960 CANDLESTICKS    






            ## BREAK & RETEST OF HISTORICAL LEVEL TEST
            ## STILL USING THE 4-HOUR AND 960 CANDLESTICKS






            ## FINAL OUTPUT - BUILD THE TABLE

                sheet = x
                sheet = sheet.replace("/", "")
                ICHMKU.append(ichmku)
                PAIR.append(x)
                RSIR.append(patRSIR)
                RSID.append(patRSID)
                RFP.append(patRFP)
                TRNGL.append(patTRNGL)



                table.to_excel(writerdft, sheet_name=sheet)
                D.index = D.index.strftime('%m %d %Y %H:%M:%S')
                D.to_excel(writerdfd, sheet_name=sheet)

    df = pd.DataFrame({'PAIR': PAIR,'ICHMKU': ICHMKU, 'RSIR': RSIR, 'RSID': RSID, 'RFP': RFP, 'TRNGL': TRNGL})
    df = df.sort_values(['ICHMKU'], ascending=False)
    df.to_excel(writerdf, sheet_name='Results')
    
print('       Finished.')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:607: RuntimeWarning: divide by zero encountered in double_scalars


       Finished.
